# 0. Discover data

In [ ]:
!pip install rasterio

In [ ]:
import pandas as pd
import rasterio
from rasterio.plot import show, show_hist
import os

import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import shutil

In [ ]:
path_metadata = '../hfactory_magic_folders/cleanr/train data/metadata.csv'
df_meta = pd.read_csv(path_metadata)
df_meta

In [ ]:
path = "../hfactory_magic_folders/cleanr/train data/images/plume/20230101_methane_mixing_ratio_id_4928.tif"
example_image = rasterio.open(path)
show(example_image, cmap="Greys", title="Satelite image of the location with ID 4928 in 2023-01-01")

In [ ]:
condition = (df_meta['id_coord'] == 'id_4928') & (df_meta['date'] == 20230101)
df_meta[condition]

In [ ]:
path = "../hfactory_magic_folders/cleanr/train data/images/no_plume/20230330_methane_mixing_ratio_id_6609.tif"
example_image = rasterio.open(path)
show(example_image, cmap="Greys", title="Satelite image of the location with ID 6609 in 2023-03-30")

In [ ]:
condition = (df_meta['id_coord'] == 'id_6609') & (df_meta['date'] == 20230330)
df_meta[condition]

# I. Data augmentation addition

In [ ]:
from PIL import Image
import os

# Directory path containing your original images
original_directory = "../hfactory_magic_folders/cleanr/train data/images/plume"

# Output directory for augmented images
output_directory = "data_augmented/plume"

# Function to rotate images and save both original and rotated versions
def augment_and_save_images(original_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)  # Create the output directory if it doesn't exist

    for filename in os.listdir(original_directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".tif"):
            # Load the original image
            original_image_path = os.path.join(original_directory, filename)
            original_image = Image.open(original_image_path)

            # Rotate the original image by 90 degrees
            rotated_image = original_image.rotate(90)
            
            # Flip the image horizontally
            flipped_image = original_image.transpose(Image.FLIP_LEFT_RIGHT)
            
            # Save the original image and the rotated image in the output directory
            original_image.save(os.path.join(output_directory, f"original_{filename}"))
            rotated_image.save(os.path.join(output_directory, f"rotated_{filename}"))
            flipped_image.save(os.path.join(output_directory, f"flipped_{filename}"))

# Call the function to augment and save images
# augment_and_save_images(original_directory, output_directory)

In [ ]:
from PIL import Image
import os

# Directory path containing your original images
original_directory = "../hfactory_magic_folders/cleanr/train data/images/no_plume"

# Output directory for augmented images
output_directory = "data_augmented/no_plume"

# Function to rotate images and save both original and rotated versions
def augment_and_save_images(original_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)  # Create the output directory if it doesn't exist

    for filename in os.listdir(original_directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".tif"):
            # Load the original image
            original_image_path = os.path.join(original_directory, filename)
            original_image = Image.open(original_image_path)

            # Rotate the original image by 90 degrees
            rotated_image = original_image.rotate(90)
            
            # Flip the image horizontally
            flipped_image = original_image.transpose(Image.FLIP_LEFT_RIGHT)
            
            # Save the original image and the rotated image in the output directory
            original_image.save(os.path.join(output_directory, f"original_{filename}"))
            rotated_image.save(os.path.join(output_directory, f"rotated_{filename}"))
            flipped_image.save(os.path.join(output_directory, f"flipped_{filename}"))

# Call the function to augment and save images
# augment_and_save_images(original_directory, output_directory)

# II. Prepare data

In [ ]:
import torch
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, Subset

In [ ]:
def is_tif(image_str):
    if image_str[-4:] == ".tif":
        return True
    else:
        return False

In [ ]:
# Define a transform to preprocess the images (you can adjust the normalization values)
transform = transforms.Compose([
    transforms.ToTensor()
])

# Path to your dataset folder
data_dir = 'data_augmented/'

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_tif)

# Extract labels from the dataset
labels = [label for _, label in dataset]

# Perform a stratified split
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, stratify=labels, random_state=42)

# Create subsets for training and test data
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders for both subsets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# III. Pipeline preparation

In [ ]:
def evaluate(model, loader):
    model.eval()
    targets = []
    #Obtain a list of prediction scores. If the prediction score >= 0.5, it means the image contains a plume, else not
    predictions = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images).squeeze()
            if outputs.cpu().size() == torch.Size([]):
                predictions += [outputs.cpu().item()]
                targets += [labels.cpu().item()]
            else:
                predictions += outputs.cpu().tolist()
                targets += labels.cpu().tolist()
    accuracy = accuracy_score(targets, [round(p) for p in predictions])
    auc_score = roc_auc_score(targets, predictions) * 100
    return accuracy, auc_score

In [ ]:
def train_model(model, train_loader, test_loader, optimizer, loss, num_epochs=10):
    # Lists to store the loss and AUC scores
    train_losses = []
    test_losses = []
    auc_scores = []
    test_acc = []

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
    
            # Ensure labels are of type torch.FloatTensor
            labels = labels.float().to(device)
    
            optimizer.zero_grad()
            outputs = model(images).squeeze()
    
            # Reshape the labels to match the shape of outputs
            labels = labels.view_as(outputs)
    
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_dataset)
    
        # Calculate test loss and metrics
        model.eval()
        test_loss = 0.0
        # Iterate through the test loader
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.float().to(device)

            # Forward pass
            with torch.no_grad():  # Ensure no gradient calculation during inference
                outputs = model(images).squeeze()
            labels = labels.view_as(outputs)

            # Calculate the loss (assuming you're using BCEWithLogitsLoss)
            loss = criterion(outputs, labels)

            # Accumulate the test loss
            test_loss += loss.item() * images.size(0)

        # Calculate the average test loss
        test_loss = test_loss / len(test_loader.dataset)
    
        test_accuracy, test_auc_score = evaluate(model, test_loader)  # Ensure evaluate function returns these metrics
    
        # Store the loss and AUC score for plotting
        train_losses.append(train_loss)
        test_losses.append(test_loss)  # Update test loss here
        auc_scores.append(test_auc_score)
        test_acc.append(test_accuracy)
    
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}, Test AUC: {test_auc_score:.2f}%")
        
    return train_losses, test_losses, auc_scores, test_acc


# IV. Simple model

In [ ]:
# Define CNN model
class PlumeCNN(nn.Module):
    def __init__(self):
        super(PlumeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(self.bn1(nn.functional.relu(self.conv1(x))))
        x = self.pool(self.bn2(nn.functional.relu(self.conv2(x))))
        x = x.view(-1, 64 * 16 * 16)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = torch.sigmoid(self.fc2(x))
        return x

In [ ]:
learning_rate = 0.001
num_epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = PlumeCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_losses, test_losses, auc_scores, test_acc = train_model(model, train_loader, test_loader, optimizer, criterion, num_epochs=50)

# V. Plot model metrics

In [ ]:
#Plotting caracteristics.

font = {'family': 'serif',
        'color':  'darkblue',
        'weight': 'normal',
        'size': 16,
        }
figure_size = (15,10)

In [ ]:
def plot_loss(train_losses, test_losses):
    list_epochs = [k for k in range(len(train_losses))]
    
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(list_epochs, train_losses, c='blue', label='Training loss')
    ax.plot(list_epochs, test_losses, c='red', label='Test loss')
    
    ax.set_title('Training and test losses', fontdict=font)
    ax.set_xlabel('Epoch', fontdict=font)
    ax.set_ylabel('Loss (BCE)', fontdict=font)
    ax.legend()
    plt.show()

In [ ]:
def plot_metric(metric, metric_name):
    list_epochs = [k for k in range(len(metric))]
    
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(list_epochs, metric, label=metric_name)
    
    ax.set_title('Metric: ' + metric_name, fontdict=font)
    ax.set_xlabel('Epoch', fontdict=font)
    ax.set_ylabel(metric_name, fontdict=font)
    ax.legend()
    plt.show()

In [ ]:
plot_loss(train_losses, test_losses)

In [ ]:
plot_metric(auc_scores, 'AUC')

In [ ]:
plot_metric(test_acc, 'Accuracy')

# VI. Compare models

In [ ]:
def plot_metric_models(metric, metric_name, list_model_names):
    """
    metric: for the same metric, list of list of metrics.
    """
    n_models = len(list_model_names)
    fig, ax = plt.subplots(figsize=(10,5))
    for k in range(n_models):
        list_epochs = [k for k in range(len(metric[k]))]
        ax.plot(list_epochs, metric[k], label=list_model_names[k])
    ax.set_title(metric_name + ' for different models', fontdict=font)
    ax.set_xlabel('Epoch', fontdict=font)
    ax.set_ylabel(metric_name, fontdict=font)
    ax.legend()
    plt.show()

In [ ]:
# Define a transform to preprocess the images (you can adjust the normalization values)
transform = transforms.Compose([
    transforms.ToTensor()
])

# Path to your dataset folder
data_dir = 'window_3/'

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_tif)

# Extract labels from the dataset
labels = [label for _, label in dataset]

# Perform a stratified split
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, stratify=labels, random_state=42)

# Create subsets for training and test data
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders for both subsets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = PlumeCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses_window_3, test_losses_window_3, auc_scores_window_3, test_acc_window_3 = train_model(model, train_loader, test_loader, optimizer, criterion, num_epochs=50)

In [ ]:
# Define a transform to preprocess the images (you can adjust the normalization values)
transform = transforms.Compose([
    transforms.ToTensor()
])

# Path to your dataset folder
data_dir = 'window_4/'

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_tif)

# Extract labels from the dataset
labels = [label for _, label in dataset]

# Perform a stratified split
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, stratify=labels, random_state=42)

# Create subsets for training and test data
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders for both subsets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = PlumeCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses_window_4, test_losses_window_4, auc_scores_window_4, test_acc_window_4 = train_model(model, train_loader, test_loader, optimizer, criterion, num_epochs=50)

In [ ]:
# Define a transform to preprocess the images (you can adjust the normalization values)
transform = transforms.Compose([
    transforms.ToTensor()
])

# Path to your dataset folder
data_dir = '../hfactory_magic_folders/cleanr/train data/images/'

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_tif)

# Extract labels from the dataset
labels = [label for _, label in dataset]

# Perform a stratified split
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, stratify=labels, random_state=42)

# Create subsets for training and test data
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders for both subsets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
model = PlumeCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses_base, test_losses_base, auc_scores_base, test_acc_base = train_model(model, train_loader, test_loader, optimizer, criterion, num_epochs=50)

In [ ]:
metric = [auc_scores_base, auc_scores, auc_scores_window_3, auc_scores_window_4]
metric_name = 'AUC score'
list_model_names = ['Base CNN', 'CNN on data augmented', 'CNN on window_3', 'CNN on window_4']
plot_metric_models(metric, metric_name, list_model_names)

In [ ]:
metric = [test_acc_base, test_acc, test_acc_window_3, test_acc_window_4]
metric_name = 'Test accuracy'
list_model_names = ['Base CNN', 'CNN on data augmented', 'CNN on window_3', 'CNN on window_4']
plot_metric_models(metric, metric_name, list_model_names)

# VII. ResNet fine-tuning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [ ]:
# Define a transform to preprocess the images (you can adjust the normalization values)
transform = transforms.Compose([
    transforms.ToTensor()
])

# Path to your dataset folder
data_dir = '../hfactory_magic_folders/cleanr/train data/images/'

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_tif)

# Extract labels from the dataset
labels = [label for _, label in dataset]

# Perform a stratified split
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, stratify=labels, random_state=42)

# Create subsets for training and test data
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders for both subsets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load a pretrained ResNet model
model = models.resnet18(pretrained=True)

# Modify the final classification layer for binary classification (1 output neuron) with sigmoid activation
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()  # Add sigmoid activation
)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Place the model on GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Done")

train_losses_resnet, test_losses_resnet, auc_scores_resnet, test_acc_resnet = train_model(model, train_loader, test_loader, optimizer, criterion, num_epochs=50)

In [ ]:
metric = [test_acc_base, test_acc_resnet]
metric_name = 'Test accuracy'
list_model_names = ['Base CNN', 'ResNet finetuned']
plot_metric_models(metric, metric_name, list_model_names)

# For later

In [ ]:
# Define a transform to preprocess the images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3-channel (RGB)
    transforms.Resize((224, 224)),  # Resize to match the input size of ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to your dataset folder
data_dir = '../hfactory_magic_folders/cleanr/train data/images/'

# Create the ImageFolder dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform, is_valid_file=is_tif)

# Split the dataset into training, validation, and test sets (adjust ratios as needed)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size])

# Define the data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)